In [2]:
from enhanced_lib.exchange import ExchangeCache,Account,Database
import asyncio

account = Account(
    'http://localhost:8000',
    "main_account",
    "main_account",
)
db = Database(
    'http://localhost:8000',
)

exchange = ExchangeCache(account, "BTCUSDT")

# await exchange.fetch_latest(False)

# exchange.open_orders.get_tp_orders('long',True)

# await account.fetch_config('BTCUSDT')
# await exchange.fetch_latest(True)
# exchange.get_next_tradable_zone("long")
# exchange.get_next_tradable_zone("short")
# exchange.get_zone_for_position('long')
# exchange.maximum_sell_size
# exchange.future_instance.config.get_trading_zones('kind')
# exchange.open_orders.get_sl_orders('short',True)
# await account.update_config_fields('BTCUSDT',{'budget':400,'max_size':0.21})
# await account.update_config_resistance('BTCUSDT')
# account.general_config['config']['BTCUSDT']
# exchange.future_instance.config.as_dict()
# future_instance = exchange.future_instance
# exchange.account.general_config['config']['BTCUSDT']
# future_instance.get_trading_zones('long')
# future_instance.config.long_liquidation_price
# future_instance.config.as_dict()
# zones = future_instance.config.get_trading_zones('short')
# zones
# stop = future_instance.config.long_liquidation_price
# stop
# app_config = future_instance.config.app_config
# app_config.entry = 44779.3
# app_config.stop = stop
# # app_config.raw = True
# # workers.determine_optimum_reward(app_config)
# exchange.maximum_sell_size
# exchange.get_zone_for_position('long')
# exchange.get_zone_for_position('long')
# exchange.get_next_tradable_zone('long')
# exchange.config_params_for_future_trades2('long',True)
await db.generate_and_save_future_trades('main_account','BTCUSDT',ignore=True)

error maximum recursion depth exceeded while calling a Python object
size for risk 6 0.0
size for risk 11 0.007
size for risk 21 0.016
size for risk 36 0.033
size for risk 56 0.055
size for risk 81 0.087
size for risk 111 0.126
size for risk 146 0.172
size for risk 186 0.232
size for risk 231 0.295
size for risk 281 0.373
error maximum recursion depth exceeded while calling a Python object
size for risk 6 0.0
size for risk 11 0.007
size for risk 21 0.016
size for risk 36 0.033
size for risk 56 0.055
size for risk 81 0.087
size for risk 111 0.126
size for risk 146 0.172
size for risk 186 0.232
size for risk 231 0.295
size for risk 281 0.372
error maximum recursion depth exceeded while calling a Python object
size for risk 6 0.0
size for risk 11 0.003
size for risk 21 0.015
size for risk 36 0.031
size for risk 56 0.053
size for risk 81 0.085
size for risk 111 0.123
size for risk 146 0.169
size for risk 186 0.225
size for risk 231 0.288
size for risk 281 0.365
error maximum recursion dept

In [8]:
exchange = await db.get_initialized_exchange('main_account','BTCUSDT','gbozee1_sub_account')

exchange.position.long
exchange.get_zone_for_position('long')

[{'stop': 41092.4604, 'entry': 41359.4, 'pnl': 11.935399968000361},
 {'stop': 41359.4, 'entry': 41626.3396, 'pnl': 37.56160156800016},
 {'stop': 41626.3396, 'entry': 42013.1758, 'pnl': 74.69787676799996},
 {'stop': 42013.1758, 'entry': 42490.5, 'pnl': 120.52099996800023}]